In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
# embeddings = OpenAIEmbeddings()

In [6]:
"""
虽然LangChain中有几种预定义的内存类型，但你很可能想要添加自己的内存类型，以便为你的应用程序提供最佳性能。本笔记将介绍如何实现此操作。

在本笔记中，我们将向ConversationChain添加自定义内存类型。为了添加自定义内存类，我们需要导入基础内存类并对其进行子类化。
"""
from typing import Any, Dict, List

from langchain.chains import ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
 

In [7]:
"""
在此示例中，我们将编写一个自定义内存类，该类使用spacy提取实体并将有关它们的信息保存在简单的哈希表中。

然后，在对话期间，我们将查看输入文本，提取任何实体，并将有关它们的任何信息放入上下文中。

请注意，此实现非常简单和脆弱，可能在生产环境中无用。它的目的是展示您可以添加自定义内存实现。
为此，我们需要使用spacy。
"""
# !pip install spacy
# !python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load('en_core_web_lg')


In [8]:
"""
TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases
"""
class SpacyEntityMemory(BaseMemory, BaseModel):
    """Memory class for storing information about entities."""

    # Define dictionary to store information about entities.
    entities: dict = {}
    # Define key to pass information about entities into prompt.
    memory_key: str = "entities"

    def clear(self):
        self.entities = {}

    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return [self.memory_key]

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""
        # Get the input text and run through spaCy
        doc = nlp(inputs[list(inputs.keys())[0]])
        # Extract known information about entities, if they exist.
        entities = [
            self.entities[str(ent)] for ent in doc.ents if str(ent) in self.entities
        ]
        # Return combined information about entities to put into context.
        return {self.memory_key: "\n".join(entities)}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save context from this conversation to buffer."""
        # Get the input text and run through spaCy
        text = inputs[list(inputs.keys())[0]]
        doc = nlp(text)
        # For each entity that was mentioned, save this information to the dictionary.
        for ent in doc.ents:
            ent_str = str(ent)
            if ent_str in self.entities:
                self.entities[ent_str] += f"\n{text}"
            else:
                self.entities[ent_str] = text

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [ ]:
"""
现在我们定义一个提示，以输入有关实体的信息以及用户输入
"""
from langchain.prompts.prompt import PromptTemplate
 
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. You are provided with information about entities the Human mentions, if relevant.
 
Relevant entity information:
{entities}
 
Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["entities", "input"], template=template
)

In [ ]:
"""
现在我们将它们组合在一起！
"""
conversation = ConversationChain(llm=llm, prompt=prompt, verbose=True, memory=SpacyEntityMemory())


In [ ]:
"""
在第一个示例中，没有关于Harrison的先前知识，"相关实体信息"部分为空。
"""
conversation.predict(input="Harrison likes machine learning")


In [ ]:
"""
现在在第二个示例中，我们可以看到它提取了有关Harrison的信息。
"""
conversation.predict(input="What do you think Harrison's favorite subject in college was?")


In [ ]:
"""
再次提醒，此实现方式相当简单且脆弱，可能在生产环境中无用。它的目的是展示您可以添加自定义内存实现。
"""